In [1]:
import matplotlib.pyplot as plt
import math
import ifcopenshell
import ifcopenshell.api.alignment
import ifcopenshell.api.unit
import ifcopenshell.geom
import numpy as np


def make_angle(slope):
    # from Fig 8.15.3.15.A, X is towards the left, Y is up, and angle is positive CW from +X.
    # this function converts tranditional engineering slopes to angular measure for use in IfcOpenCrossProfileDef
    return math.pi + math.atan(slope)

def create_superelevation_event(file,name,alignment,dist_along,side,slope):
    curve = ifcopenshell.api.alignment.get_curve(alignment)
    referent = file.createIfcReferent(
        GlobalId=ifcopenshell.guid.new(),
        OwnerHistory=None,
        Name=name,
        Description=None,
        ObjectType=None,
        ObjectPlacement=file.createIfcLinearPlacement(
            RelativePlacement=file.createIfcAxis2PlacementLinear(
                Location=file.createIfcPointByDistanceExpression(DistanceAlong=file.createIfcLengthMeasure(dist_along),BasisCurve=curve)
            )
        ),
        Representation=None,
        PredefinedType="SUPERELEVATIONEVENT",
    )
    pset_superelevation = ifcopenshell.api.pset.add_pset(file, product=referent, name="Pset_Superelevation")
    #NOTE: Side and TransitionSuperelevation are PEnum_ properties. Notice the [] to get them to work out correctly.
    ifcopenshell.api.pset.edit_pset(file, pset=pset_superelevation, properties={"Side": [side],"Superelevation":slope,"TransitionSuperelevation":["LINEAR"]})
    
    nest = ifcopenshell.api.alignment.get_referent_nest(file, alignment)
    nest.RelatedObjects += (referent,)
    return referent


def append_supertransition(file,alignment,surface,start,runout,runoff,full,crownslope,fullsuper,width,pivot=1):
    start_station = ifcopenshell.api.alignment.get_alignment_start_station(file,alignment)
    
    r1 = create_superelevation_event(file,f"Start Tangent Runout ({ifcopenshell.util.alignment.station_as_string(file,start_station+start)})",alignment,start,"LEFT" if fullsuper < 0. else "RIGHT",-crownslope if fullsuper < 0. else crownslope)
    r2 = create_superelevation_event(file,f"Start Tangent Runoff ({ifcopenshell.util.alignment.station_as_string(file,start_station+start+runout)})",alignment,start+runout,"LEFT" if fullsuper < 0. else "RIGHT",-crownslope if fullsuper < 0. else crownslope)
    r3 = create_superelevation_event(file,"CC",alignment,start+2*runout,"BOTH",-crownslope if fullsuper < 0. else crownslope)
    r4 = create_superelevation_event(file,f"Start Full Superelevation ({ifcopenshell.util.alignment.station_as_string(file,start_station+start+runout+runoff)})",alignment,start+runout+runoff,"BOTH",fullsuper)
    r5 = create_superelevation_event(file,f"End Full Superelevation ({ifcopenshell.util.alignment.station_as_string(file,start_station+start+runout+runoff+full)})",alignment,start+runout+runoff+full,"BOTH",fullsuper)
    r6 = create_superelevation_event(file,"CC",alignment,start+runout+runoff+full+runoff-runout,"BOTH",-crownslope if fullsuper < 0. else crownslope)
    r7 = create_superelevation_event(file,f"End Tangent Runoff ({ifcopenshell.util.alignment.station_as_string(file,start_station+start+runout+runoff+full+runoff)})",alignment,start+runout+runoff+full+runoff,"LEFT" if fullsuper < 0. else "RIGHT",-crownslope if fullsuper < 0. else crownslope)
    r8 = create_superelevation_event(file,f"End Tangent Runout ({ifcopenshell.util.alignment.station_as_string(file,start_station+start+runout+runoff+full+runoff+runout)})",alignment,start+runout+runoff+full+runoff+runout,"LEFT" if fullsuper < 0. else "RIGHT",-crownslope if fullsuper < 0. else crownslope)

    # assumes pivot at CL
    op1 = file.createIfcCartesianPoint((width,-width*crownslope))
    op2 = file.createIfcCartesianPoint((width,-width*crownslope)) if fullsuper < 0. else file.createIfcCartesianPoint((width,0.0))
    op3 = file.createIfcCartesianPoint((width,-width*crownslope)) if fullsuper < 0. else file.createIfcCartesianPoint((width,width*crownslope))
    op4 = file.createIfcCartesianPoint((width,width*fullsuper))
        
    cs1 = file.createIfcOpenCrossProfileDef(
        ProfileType="CURVE",
        HorizontalWidths=True,
        Widths=[width,width],
        Slopes=[make_angle(-crownslope),make_angle(crownslope)],
        OffsetPoint=op1
    )

    cs2 = file.createIfcOpenCrossProfileDef(
        ProfileType="CURVE",
        HorizontalWidths=True,
        Widths=[width,width],
        Slopes = [make_angle(-crownslope),make_angle(0.0)] if fullsuper < 0. else [make_angle(0.0),make_angle(crownslope)],
        OffsetPoint=op2
    )

    cs3 = file.createIfcOpenCrossProfileDef(
        ProfileType="CURVE",
        HorizontalWidths=True,
        Widths=[width,width],
        Slopes=[make_angle(-crownslope),make_angle(-crownslope)] if fullsuper < 0. else [make_angle(crownslope),make_angle(crownslope)],
        OffsetPoint=op3
    )

    cs4 = file.createIfcOpenCrossProfileDef(
        ProfileType="CURVE",
        HorizontalWidths=True,
        Widths=[width,width],
        Slopes=[make_angle(fullsuper),make_angle(fullsuper)],
        OffsetPoint=op4
    )

    surface.CrossSectionPositions += (r1.ObjectPlacement.RelativePlacement,r2.ObjectPlacement.RelativePlacement,r3.ObjectPlacement.RelativePlacement,r4.ObjectPlacement.RelativePlacement,r5.ObjectPlacement.RelativePlacement,r6.ObjectPlacement.RelativePlacement,r7.ObjectPlacement.RelativePlacement,r8.ObjectPlacement.RelativePlacement)
    surface.CrossSections += (cs1,cs2,cs3,cs4,cs4,cs3,cs2,cs1)

    file.createIfcRelAssociatesProfileDef(GlobalId=ifcopenshell.guid.new(),RelatedObjects=(r1,r8),RelatingProfileDef=cs1)
    file.createIfcRelAssociatesProfileDef(GlobalId=ifcopenshell.guid.new(),RelatedObjects=(r2,r7),RelatingProfileDef=cs2)
    file.createIfcRelAssociatesProfileDef(GlobalId=ifcopenshell.guid.new(),RelatedObjects=(r3,r6),RelatingProfileDef=cs3)
    file.createIfcRelAssociatesProfileDef(GlobalId=ifcopenshell.guid.new(),RelatedObjects=(r4,r5),RelatingProfileDef=cs4)


file = ifcopenshell.file(schema="IFC4X3_ADD2")
project = file.createIfcProject(GlobalId=ifcopenshell.guid.new(),Name="Superelevation Example")
site = file.createIfcSite(GlobalId=ifcopenshell.guid.new(),Name="Site")

length = ifcopenshell.api.unit.add_conversion_based_unit(file,name="foot")
angle = ifcopenshell.api.unit.add_si_unit(file,unit_type="PLANEANGLEUNIT")
ifcopenshell.api.unit.assign_unit(file,units=[length,angle])
geometric_representation_context = ifcopenshell.api.context.add_context(file, context_type="Model")
axis_model_representation_subcontext = ifcopenshell.api.context.add_context(
    file,
    context_type="Model",
    context_identifier="Axis",
    target_view="MODEL_VIEW",
    parent=geometric_representation_context,
)
body = ifcopenshell.api.context.add_context(
    file,
    context_type="Model", 
    context_identifier="Body", 
    target_view="MODEL_VIEW", 
    parent=geometric_representation_context)

ifcopenshell.api.aggregate.assign_object(file,relating_object=project,products=[site,])

start_station = 500.
alignment = ifcopenshell.api.alignment.create(file,"A-Line",include_vertical=True,start_station=start_station)
layout = ifcopenshell.api.alignment.get_horizontal_layout(alignment)

segment1 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint(Coordinates=((500.,2500.))),
    StartDirection=math.radians(327.0613),
    StartRadiusOfCurvature=0.0,
    EndRadiusOfCurvature=0.0,
    SegmentLength=1956.785654,
    PredefinedType = "LINE"
)

end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment1)

unit_scale = ifcopenshell.util.unit.calculate_unit_scale(file)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
dir = math.atan2(dy,dx)
segment2 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint((x,y)),
    StartDirection=dir,
    StartRadiusOfCurvature=1000.,
    EndRadiusOfCurvature=1000.,
    SegmentLength=1919.222667,
    PredefinedType="CIRCULARARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment2)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
dir = math.atan2(dy,dx)
segment3 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint((x,y)),
    StartDirection=dir,
    StartRadiusOfCurvature=0.0,
    EndRadiusOfCurvature=0.0,
    SegmentLength=1886.905454,
    PredefinedType = "LINE"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment3)


x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
dir = math.atan2(dy,dx)
segment4 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint((x,y)),
    StartDirection=dir,
    StartRadiusOfCurvature=-1250.,
    EndRadiusOfCurvature=-1250.,
    SegmentLength=1848.115835,
    PredefinedType="CIRCULARARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment4)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
dir = math.atan2(dy,dx)
segment5 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint((x,y)),
    StartDirection=dir,
    StartRadiusOfCurvature=0.0,
    EndRadiusOfCurvature=0.0,
    SegmentLength=1564.635765,
    PredefinedType = "LINE"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment5)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
dir = math.atan2(dy,dx)
segment6 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint((x,y)),
    StartDirection=dir,
    StartRadiusOfCurvature=-950.,
    EndRadiusOfCurvature=-950.,
    SegmentLength=1049.119737,
    PredefinedType="CIRCULARARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment6)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
dir = math.atan2(dy,dx)
segment7 = file.createIfcAlignmentHorizontalSegment(
    StartPoint=file.createIfcCartesianPoint((x,y)),
    StartDirection=dir,
    StartRadiusOfCurvature=0.0,
    EndRadiusOfCurvature=0.0,
    SegmentLength=2112.285084,
    PredefinedType = "LINE"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,layout,segment7)


vlayout = ifcopenshell.api.alignment.get_vertical_layout(alignment)


segment1 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=0.,
    HorizontalLength=1200.,
    StartHeight=100.,
    StartGradient=1.75/100.,
    EndGradient=1.75/100.,
    PredefinedType = "CONSTANTGRADIENT"
)

end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment1)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment2 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=1600.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=-1./100.,
    PredefinedType = "PARABOLICARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment2)


x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment3 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=1600.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=-1./100.,
    PredefinedType = "CONSTANTGRADIENT"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment3)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment4 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=1200.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=2./100.,
    PredefinedType = "PARABOLICARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment4)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment5 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=800.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=2./100.,
    PredefinedType = "CONSTANTGRADIENT"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment5)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment6 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=2000.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=-2./100.,
    PredefinedType = "PARABOLICARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment6)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment7 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=1000.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=-2./100.,
    PredefinedType = "CONSTANTGRADIENT"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment7)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment8 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=800.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=-0.5/100.,
    PredefinedType = "PARABOLICARC"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment8)

x = float(end[0,3])/unit_scale
y = float(end[1,3])/unit_scale
dx = float(end[0,0])
dy = float(end[1,0])
segment9 = file.createIfcAlignmentVerticalSegment(
    StartDistAlong=x,
    HorizontalLength=2600.,
    StartHeight=y,
    StartGradient=dy/dx,
    EndGradient=-0.5/100.,
    PredefinedType = "CONSTANTGRADIENT"
)
end = ifcopenshell.api.alignment.create_layout_segment(file,vlayout,segment9)

road = file.createIfcRoad(GlobalId=ifcopenshell.guid.new(),Name="Road1")
ifcopenshell.api.aggregate.assign_object(file,relating_object=site,products=[road,])

road_part = file.createIfcRoadPart(GlobalId=ifcopenshell.guid.new(),Name="RoadPart1",UsageType="LONGITUDINAL")
ifcopenshell.api.aggregate.assign_object(file,relating_object=road,products=[road_part,])

crownslope = 0.2
fullsuper = 0.6
width = 30.
    
nest = ifcopenshell.api.alignment.get_referent_nest(file,alignment)
ref = nest.RelatedObjects[0]
surface = file.createIfcSectionedSurface(
    Directrix = ifcopenshell.api.alignment.get_curve(alignment),
    CrossSectionPositions=[ref.ObjectPlacement.RelativePlacement],
    CrossSections=[file.createIfcOpenCrossProfileDef(
        ProfileType="CURVE",
        HorizontalWidths=True,
        Widths=[width,width],
        Slopes=[make_angle(-crownslope),make_angle(crownslope)],
        OffsetPoint=file.createIfcCartesianPoint((width,-width*crownslope))
    )
]
)

append_supertransition(file,alignment,surface,2400-start_station,200,500,700,crownslope,-fullsuper,width)
append_supertransition(file,alignment,surface,6200-start_station,200,500,700,crownslope,fullsuper,width)
append_supertransition(file,alignment,surface,9600-start_station,200,500,700,crownslope,fullsuper,width)


representation = file.createIfcShapeRepresentation(
    ContextOfItems=body, RepresentationIdentifier="Body", RepresentationType="SectionedSurface", Items=[surface])


product_rep = file.createIfcProductDefinitionShape(Representations=[representation])


pavement = file.createIfcPavement(GlobalId=ifcopenshell.guid.new(),Name="Pavement",
        ObjectPlacement=file.createIfcLocalPlacement(
            RelativePlacement=file.createIfcAxis2Placement3D(
                Location=file.createIfcCartesianPoint(Coordinates=((0.0,0.0,0.0)))
            )
        ),
        #ObjectPlacement=file.createIfcLinearPlacement(
        #    RelativePlacement=file.createIfcAxis2PlacementLinear(
        #        Location=file.createIfcPointByDistanceExpression(DistanceAlong=file.createIfcLengthMeasure(0.0),BasisCurve=curve)
        #    )
        #),
        Representation=product_rep
)

ifcopenshell.api.spatial.assign_container(file,relating_structure=road_part,products=[pavement,])


#file.write("C:/Users/bricer/OneDrive - Washington State Department of Transportation/Desktop/Superelevation.ifc")
file.write(r"C:\Users\rickb\OneDrive\Desktop\SuperTransition.ifc")
print("Done!")

Done!
